In [1]:
import os 
os.chdir("..")

In [2]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
from scipy import stats
import json
import math

from utils import *
from config import DEVICE_SKIP_STATE, NUMERIC_CTX_SUFFIX, DEVICE_MULTI_STATE_SUFFIX
root_dir = "data/Thermostat Dataset"
processed_dir = "processed"
datetime_temp = "%Y-%m-%d %H:%M"
all_csv = [f for f in os.listdir(root_dir) if f.endswith(".csv") \
            and ("weather" not in f) and ("RB" not in f)]
weather_file = "weather.csv"
weathers = pd.read_csv(os.path.join(root_dir, weather_file))
# indoor_temp_matching = lambda x : any(["RMT_TL", ""])
col_idx = {
    "Time": 0,
    "Window": 1,
    "Occupancy": 2,
    "IndoorTemp": 3,
    "IndoorHumidity": 4,
    "IndoorCo2": 5,
    "Setpoint": 6,
    "OutTemp": 7,
}
ctx_to_include = [
    "OutTemp", "IndoorTemp", "IndoorHumidity",
]
outtemp_data = weathers.iloc[:,1]
print(min(outtemp_data), max(outtemp_data))
setpoint_set = set()
# all_csv = ['HS3313.csv', 'HS5314.csv']
test_freq = {}
for filename in all_csv:
    file_path = os.path.join(root_dir, filename)
    data = pd.read_csv(file_path)
    data["OutTemp"] = outtemp_data
    occupied_idx = data.iloc[:, col_idx["Occupancy"]] >= 1
    print(filename, len(data[occupied_idx]))
    # setpoint_data = data.iloc[:, col_idx["Setpoint"]][occupied_idx]
    # humid_data = data.iloc[:, col_idx["IndoorHumidity"]][occupied_idx]
    # indoorTemp_data = data.iloc[:, col_idx["IndoorTemp"]][occupied_idx]

    # device_evt = {"thermostat":[]}
    # ctx_evt = {
    #     x:[]
    #     for x in ctx_to_include
    # }
    # prev_setpoint = None
    # prev_state = DEVICE_SKIP_STATE
    # setpoint_this_user = set(data.iloc[:, col_idx["Setpoint"]])
    # setpoint_outTemp = {
    #     x:[0,0]
    #     for x in setpoint_this_user
    # }
    # setpoint_set = setpoint_set.union(setpoint_this_user)
    # for idx, row in data.iterrows():
    #     setpoint = float(row[col_idx["Setpoint"]])
    #     if setpoint == 0:
    #         print(row)
    #     setpoint_outTemp[setpoint][0] += 1
    #     setpoint_outTemp[setpoint][1] += row["OutTemp"]
    #     time = row[col_idx["Time"]]
    #     cur_time = datetime.strptime(time, datetime_temp)
    #     occupancy = row[col_idx["Occupancy"]]
    #     if occupancy == 0:
    #         # No one in the room
    #         if prev_state != DEVICE_SKIP_STATE:
    #             device_evt["thermostat"].append((DEVICE_SKIP_STATE, cur_time))
    #             prev_state = DEVICE_SKIP_STATE
    #             prev_setpoint = None
    #     else:
    #         if prev_setpoint != setpoint:
    #             prev_state = "auto" + DEVICE_MULTI_STATE_SUFFIX
    #             prev_setpoint = setpoint
    #             device_evt["thermostat"].append((prev_state, cur_time, setpoint))

    #     # Test part
    #     if row["OutTemp"]>=0 and row["OutTemp"] <5 and\
    #         datetime_to_mins(cur_time) >=300 and datetime_to_mins(cur_time) < 360:
    #         if occupancy == 0:
    #             setpoint = None
    #         if cur_time in test_freq:
    #             test_freq[cur_time].append(setpoint)
    #             # test_freq[cur_time].append(row[col_idx["IndoorHumidity"]])
    #         else:
    #             test_freq[cur_time] = [setpoint]
    # # print({k:v[1]/v[0] for k,v in setpoint_outTemp.items()})
    # time_data = list(data.iloc[:, col_idx["Time"]])
    # for ctx in ctx_to_include:
    #     data_col = list(data.iloc[:, col_idx[ctx]])
    #     for i, value in enumerate(data_col):
    #         if len(ctx_evt[ctx]) == 0 or \
    #             value != ctx_evt[ctx][-1][0]:
    #             cur_time = datetime.strptime(time_data[i], datetime_temp)
    #             ctx_evt[ctx].append((value, cur_time))
    

    # ctx_evt = { k + NUMERIC_CTX_SUFFIX : v for k,v in ctx_evt.items()}
    # with open(os.path.join(root_dir, processed_dir,  filename), 'w') as f:
    #     f.write(json.dumps((ctx_evt, device_evt), default=str))
    # print(device_evt)

# cnt = 0.0
# tot = 0
# for d in sorted(test_freq.keys()):
#     if test_freq[d][0] == None or test_freq[d][1] == None:
#         continue
#     print(d, test_freq[d])
#     tot += 1
#     if test_freq[d][0] != test_freq[d][1]:
#         cnt += 1.
# print(cnt / tot)
# print(sorted(list(setpoint_set)))

-23.84985161 33.6
HS3301.csv 11971
HS3307.csv 5716
HS3313.csv 13561
HS3310.csv 5492
HS3304.csv 3459
HS3311.csv 2519
HS5303.csv 4502
HS5302.csv 4672
HS5314.csv 15548
HS5305.csv 4884
HS5304.csv 4408
HS5309.csv 11682
HS3308.csv 6478
HS3309.csv 6606


In [ ]:
HS3301.csv
HS3313.csv
HS5314.csv
HS5309.csv
